# First tests with Roberta on agnews

In [ ]:
import os

# CHANGE WORKING DIRECTORY TO ROOT
current_dir = os.path.basename(os.getcwd())
if current_dir == "src":
    os.chdir("..") # Move up by 1
elif os.path.basename(os.getcwd()) == "bai-thesis-nlp":  
    pass # If already at root, stay there
else:
    os.chdir("../..") # Move up by 2 otherwise
    
import pandas as pd
from src._utils._helpers import get_generated_examples_df
from src._utils._run_multiclassRoBERTA import main_multiclassRoBERTA
from sklearn.model_selection import train_test_split 

In [ ]:
real_train_df = pd.read_csv("real_data/train/agnewstrainAll.csv").rename(columns={"2": "text", "3": "label"})
real_train_df.drop(columns=["0", "1"], inplace=True)

# Take 500 samples for dev set
real_train_df, dev_df = train_test_split(real_train_df, test_size=500, random_state=42, stratify=real_train_df["label"])

# synthetic data
syn_generic_df, _ = get_generated_examples_df("synthetic_data/datasets/syn_agnews_baseline_500.json")
syn_targeted_df, _ = get_generated_examples_df("synthetic_data/datasets/syn_agnews_targeted+tags_500.json")
syn_targeted_df = syn_targeted_df.drop(columns=["phenomena"])

## 1. 500 Real

In [ ]:
results = []
train_details = main_multiclassRoBERTA(
    real_df=real_train_df,
    synth_df=None,
    dev_df=dev_df,
    synth_ratio=0.0,
    max_samples=500,
    output_dir="src/agnews/experiments/exp_500real",
    log_dir="src/agnews/experiments/RoBERTA_log.json",
    generation_method=None,
)
display(train_details)

res = train_details["metrics_dev"]
res["method"] = "500real"
results.append(res)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Accuracy,F1 Macro
1,1.181100,0.749316,0.800000,0.800000,0.606807
2,0.440800,0.336836,0.896000,0.896000,0.824244
3,0.175800,0.301661,0.920000,0.920000,0.875554
4,0.079600,0.382388,0.906000,0.906000,0.859531
5,0.029900,0.447044,0.904000,0.904000,0.858877
6,0.017400,0.449899,0.916000,0.916000,0.867496
7,0.018600,0.466036,0.908000,0.908000,0.863223
8,0.013800,0.461195,0.910000,0.910000,0.865630


💾 experiment saved to: src/agnews/experiments/exp_real_only
{'accuracy': '0.9200', 'precision': '0.9175', 'recall': '0.9200', 'f1': '0.9158'}


{'experiment_name': 'exp_real_only',
 'experiment_dir': 'src/agnews/experiments/exp_real_only',
 'generation_method': None,
 'timestamp': '2025-03-16T00:48:45.082964',
 'model': 'roberta-base',
 'train_size': 500,
 'dev_size': 500,
 'synthetic_ratio': 0.0,
 'train_time_seconds': 102.648193359375,
 'metrics_dev': {'accuracy': 0.92,
  'precision': 0.9175385836773579,
  'recall': 0.92,
  'f1': 0.9158177814691767}}

## 2. 250 Real + 250 Generic Augmentation

In [ ]:
train_details = main_multiclassRoBERTA(
    real_df=real_train_df,
    synth_df=syn_generic_df, # Generic Augmentation
    dev_df=dev_df,
    synth_ratio=0.5, # 50% of the data is synthetic
    max_samples=500,
    output_dir="src/agnews/experiments/exp_250real_250generic",
    log_dir="src/agnews/experiments/RoBERTA_log.json",
    generation_method="generic augmentation",
)
display(train_details)

res = train_details["metrics_dev"]
res["method"] = "250real_250generic"
results.append(res)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Accuracy,F1 Macro
1,1.231800,0.719169,0.814000,0.814000,0.644890
2,0.536000,0.359767,0.884000,0.884000,0.826884
3,0.352000,0.391397,0.866000,0.866000,0.810246
4,0.226500,0.357997,0.898000,0.898000,0.844499
5,0.162300,0.365790,0.904000,0.904000,0.862773
6,0.119900,0.401643,0.904000,0.904000,0.858529
7,0.105700,0.401260,0.900000,0.900000,0.853468
8,0.081900,0.410169,0.900000,0.900000,0.851410


💾 experiment saved to: src/agnews/experiments/exp_random_aug_50
{'accuracy': '0.9040', 'precision': '0.9045', 'recall': '0.9040', 'f1': '0.9042'}


{'experiment_name': 'exp_random_aug_50',
 'experiment_dir': 'src/agnews/experiments/exp_random_aug_50',
 'generation_method': 'generic augmentation',
 'timestamp': '2025-03-16T00:50:34.463362',
 'model': 'roberta-base',
 'train_size': 500,
 'dev_size': 500,
 'synthetic_ratio': 0.5,
 'train_time_seconds': 101.09387874603271,
 'metrics_dev': {'accuracy': 0.904,
  'precision': 0.9044560352560352,
  'recall': 0.904,
  'f1': 0.9042082137853499}}

## 3. 250 Real + 250 Targeted Augmentation

In [ ]:
train_details = main_multiclassRoBERTA(
    real_df=real_train_df,
    synth_df=syn_targeted_df, # Targeted Augmentation
    dev_df=dev_df,
    synth_ratio=0.5, # 50% of the data is synthetic
    max_samples=500,
    output_dir="src/agnews/experiments/exp_250real_250targeted",
    log_dir="src/agnews/experiments/RoBERTA_log.json",
    generation_method="targeted augmentation",
)
display(train_details)

res = train_details["metrics_dev"]
res["method"] = "250real_250targeted"
results.append(res)

## 4. 500 Generic Augmentation

In [ ]:
train_details = main_multiclassRoBERTA(
    real_df=None,
    synth_df=syn_generic_df, # Generic Augmentation
    dev_df=dev_df,
    synth_ratio=1.0, # 100% of the data is synthetic
    max_samples=500,
    output_dir="src/agnews/experiments/exp_500generic",
    log_dir="src/agnews/experiments/RoBERTA_log.json",
    generation_method="generic augmentation",
)
display(train_details)

res = train_details["metrics_dev"]
res["method"] = "500generic"
results.append(res)

## 5. 500 Targeted Augmentation

In [ ]:
train_details = main_multiclassRoBERTA(
    real_df=None,
    synth_df=syn_targeted_df, # Targeted Augmentation
    dev_df=dev_df,
    synth_ratio=1.0, # 100% of the data is synthetic
    max_samples=500,
    output_dir="src/agnews/experiments/exp_500targeted",
    log_dir="src/agnews/experiments/RoBERTA_log.json",
    generation_method="targeted augmentation",
)
display(train_details)

res = train_details["metrics_dev"]
res["method"] = "500targeted"
results.append(res)

## Results

In [ ]:
df_results = pd.DataFrame(results)
new_order = ["method"] + [col for col in df_results.columns if col != "method"]
df_results = df_results[new_order]
df_results.to_csv("src/agnews/experiments/results_RoBERTA.csv", index=False)
display(df_results)